In [ ]:
#phone_no extraction.py
#test case
#RUNNING FINE
#import utils
import re
import nltk
import unicodedata
#from req_intl.modules.extraction import helpers
from urlextract import URLExtract
from nltk.tokenize import MWETokenizer
from tika import parser
from nltk.tokenize import wordpunct_tokenize
import tika
import os
from tika import unpack
import pandas as pd
from fuzzywuzzy import fuzz
import numpy as np



#ph_num=phone_no_extracted[0]['inference']
def clean_bookmark_footer_tag(txt):
    if type(txt) == list:
        # Remove bookmark
        txt_trans = [re.sub('bookmark: ', 'bookmark:', item) for item in txt]
        txt_trans = [re.sub('(bookmark:)+[_]', '[bookmark:]', item) for item in txt_trans]
        txt_trans = [re.sub('(bookmark:\])+\w+[\]]', '[bookmark:]', item) for item in txt_trans]
        txt_trans = [re.sub("[\[]{1,}", '[' , item) for item in txt_trans]
        txt_trans = [re.sub("(\[bookmark:)+\w+(])|(\[bookmark:])|(bookmark:)+\w+\s", '' , item) for item in txt_trans]
        # Strips multi whte spaces into one
        txt_trans = [re.sub(' +', ' ', item).strip() for item in txt_trans]
        return txt_trans
    else:
        # Many sections have bookmark in them
        txt_trans = re.sub('bookmark: ', 'bookmark:', txt)
        txt_trans = re.sub('(bookmark:)+[_]', '[bookmark:]', txt_trans)
        txt_trans = re.sub('(bookmark:\])+\w+[\]]', '[bookmark:]', txt_trans)
        txt_trans = re.sub("[\[]{1,}", '[' , txt_trans)
        txt_trans = re.sub("(\[bookmark:)+\w+(])|(\[bookmark:])|(bookmark:)+\w+\s", '' , txt_trans)
        txt_trans = re.sub(' +', ' ', txt_trans).strip()
        return txt_trans


def clean_multi_spaces_alphanum(txt):
    if type(txt) == list:
        # This cleans any leading and trailing junk. Excludes the characters given in []
        txt_trans = [re.sub(r'^[^a-zA-Z0-9“”•]{1,}|[^a-zA-Z0-9.)(“”]{1,}$',r'', item) for item in txt]
        # This removes ending with ()
        txt_trans = [re.sub(r'\(\)|\( +\)',r'', item) for item in txt_trans]
        # .net developer is causing issues
        txt_trans = [('.' + item) if item.lower().find('net ') == 0 else item for item in txt_trans]
        # Strips multi whte spaces into one
        txt_trans = [re.sub(' +', ' ', item).strip() for item in txt_trans]

        return txt_trans
    else:
        txt_trans = re.sub(r'^[^a-zA-Z0-9“”•]{1,}|[^a-zA-Z0-9.)(“”]{1,}$',r'', txt)
        txt_trans = re.sub(r'\(\)|\( +\)',r'', txt_trans)
        # .net developer is causing issues
        if txt_trans.lower().find('net ') == 0:
            txt_trans = '.' + txt_trans
        # Strips multi whte spaces into one
        txt_trans = re.sub(' +', ' ', txt_trans).strip()

        return txt_trans


def compare_certificate(resume_name,certificate):
    for i in df_csv.itertuples():
        if i._1 == resume_name:
            pd_certificate =str(i.Certification)
            match_certificate = fuzz.ratio(pd_certificate , certificate)
        #    print("\n COMPARE CERTIFICATE:")
        #    print("extracted :\t"+certificate)
        #    print("csv_certificate:"+pd_certificate)
         #   print("fuzzy_match:\t"+str(match_certificate))
          # print(str(resume_name)+"\t"+pd_certificate+"\t"+str(certificate))
            if (pd_certificate==certificate):
                return 1
            elif match_certificate>80:
                return 1
            elif (certificate == '' and pd_certificate=='nan'):
                return 1
            elif (certificate == '' and pd_certificate=='None'):
                return 1
            elif(len(certificate)==0 and len(pd_certificate)!=0): #extracted phone no is empty but value exists in csv
                return 0
            else :
                return 0



def compare_phone(resume_name,ph_num_extract):
    ph=ph_num_extract
    ph_num= "".join(c for c in ph if c not in ('!','.',':',"-"," "))
  #  print(ph_num)
       # print(j)

    for i in df_csv.itertuples():


        if i._1== resume_name:
          #  print('name')
            pd_phone=str(i.Phone)
            pd_phone_no = "".join(c for c in pd_phone if c not in ('!','.',':',"-"," "))

            #  print("yes")
          # print(pd_phone_no)
          # print(ph_num)
            match_phone_no=fuzz.ratio(pd_phone_no,ph_num)
        #    print("phone no :\t"+"\t"+pd_phone_no+"\t"+str(ph_num))
         #   print(match)
            if(pd_phone_no==ph_num): # direct match
                return 1
            elif match_phone_no>85:   #similarity match
                return 1
            # print("Extracted from resume:\t"+ph_num)
             #  print("percentage of matching\t:"+match)
            elif (ph_num=='' and pd_phone_no=='nan'): #both are empty values
                return 1
            elif(len(ph_num)==0 and len(pd_phone_no)!=0): #extracted phone no is empty but value exists in csv
                return 0
            else:
                return 0



def compare_email(resume_name,email_extract):
    email=email_extract.lower()
    for i in df_csv.itertuples():
        if i._1==resume_name:
            pd_email=str(i.Email)
            csv_email=pd_email.lower()
            match_email=fuzz.ratio(email,csv_email)
          #  print("email:\t"+"\t"+csv_email+"\t"+str(email))

            if(csv_email== email):
                return 1
            elif match_email > 85:
                return 1
            elif (email=='' and csv_email=='nan' ): #both are empty values
                return 1
            elif (email=='' and csv_email=='none'):
                return 1
            elif(len(email)==0 and len(csv_email)!=0): #extracted phone no is empty but value exists in csv
                return 0
            else:
                return 0

def compare_linked_in(resume_name,linked_in):

    for i in df_csv.itertuples():


        if i._1==resume_name:
            if linked_in != []:
                linked_in_extract = linked_in[0]['inference']
                pd_linked_in=str(i.LinkedIn)
                csv_linked_in=pd_linked_in.lower()
                match_linked_in=fuzz.ratio(csv_linked_in,linked_in_extract)
                if(csv_linked_in == linked_in_extract):
                    return 1
                elif match_linked_in > 85:
                    return 1
                else:
                    return 0
              #  print("linked_in:\t"+"\t"+csv_linked_in+"\t"+str(linked_in_extract))
        if i._1==resume_name:
            if linked_in == []:

                pd_linked_in=str(i.LinkedIn)
                csv_linked_in=pd_linked_in.lower()

                if (linked_in ==[] and csv_linked_in =='nan'):
                    return 1
                elif(linked_in == [] and len(csv_linked_in)!=0):
                    return 0
                else:
                    return 0
              #  print("linked_in:\t"+"\t"+csv_linked_in+"\t"+str(linked_in))

def resume_output_cleaner(text_content, field_name):
    if text_content:

        if field_name == "candidate_name":
            #text_content = ' '.join(pattern.findall(text_content))
            text_content = clean_multi_spaces_alphanum(text_content)
            text_content = text_content[:100]
            return text_content.title()

        elif field_name == "candidate_email":
            # text_content = str(re.match('\w ')).strip()
            text_content = text_content[:50]
            return text_content

        elif field_name == "candidate_phone_number":
            text_content = text_content[:50]
            return text_content

        # Error: Do not remove unicode in address
        elif field_name == "candidate_address":
            # text_content = ' '.join(pattern.findall(text_content))
            text_content = text_content[:200]
            return text_content


        elif field_name == "linkedin":
            text_content = text_content[:100]
            return text_content

        elif field_name == "summary":
            return clean_multi_spaces_alphanum(clean_bookmark_footer_tag(text_content))


        elif field_name == "critical_skill_matrix":
            return clean_bookmark_footer_tag(text_content)


        elif field_name == "certificate":
            return clean_multi_spaces_alphanum(clean_bookmark_footer_tag(text_content))


        elif field_name == "skill_matrix":
            return clean_multi_spaces_alphanum(clean_bookmark_footer_tag(text_content))


        elif field_name == "education":
            return clean_multi_spaces_alphanum(clean_bookmark_footer_tag(text_content))


        elif field_name == "client_name":
            #text_content = ' '.join(pattern.findall(text_content))
            text_content = clean_multi_spaces_alphanum(clean_bookmark_footer_tag(text_content))
            text_content = text_content[:100]
            return text_content


        elif field_name == "location":
            #text_content = ' '.join(pattern.findall(text_content))
            if ',' in text_content:
                text_content = re.sub(r',', r', ', text_content)
            text_content = clean_multi_spaces_alphanum(clean_bookmark_footer_tag(text_content))
            text_content = text_content[:70]
            return text_content


        elif field_name == "role":
            text_content = clean_bookmark_footer_tag(clean_multi_spaces_alphanum(text_content))
            text_content = text_content[:100]

            if any(ch in text_content for ch in ['/',',','-']):
                return text_content
            else:
                return case_formatting(text_content)


        elif field_name == "duration":
            text_content = clean_bookmark_footer_tag(clean_multi_spaces_alphanum(text_content))

            #print("Actual Date: " + text_content)
            text_content = date_standardization(text_content)
            #print("Standardized Date: " + text_content)

            text_content = text_content[:50]
            return text_content


        elif field_name == "responsibilities":
            text_content = clean_bookmark_footer_tag(clean_multi_spaces_alphanum(text_content))
            return text_content


        elif field_name == "environment":
            text_content = clean_bookmark_footer_tag(clean_multi_spaces_alphanum(text_content))
            return text_content
    else:
        return text_content






def get_cleaned_encoded_text_from_jd(text_content):

    ''' input type - str
        output type - list '''

    text = []

    text_content = re.sub(r'\[\w*:?\s*.*\]','',str(text_content))   #[helps in eliminating [boundary_ image] images ,bookmarks and images

    for line in text_content.split('\n'):
        line = unicodedata.normalize("NFKD",line)  #replaces \xa0 with space
        line = line.replace('–','-').encode('ascii','ignore').decode('utf-8').strip()    # encoding the line ,to remove unicode characters
        if line != '':  #after encoding checking if its empty line
            line = re.sub(r'[\t\r\f\v\s]{1,}',' ',line)        #removing all types of spaces
            text.append(line)

    return text
def get_first_block_from_resume(text_content):


    text_content = re.sub(r'\[\w*:?\s*.*\]','',str(text_content)) #boundary images
    exclude_list = ['key accomplishments', 'employment','professional summary' 'scholastics', 'online portfolio', 'declaration', 'academia', 'education/ certification', 'work experience', 'references', 'experience', 'tools and technologies', 'skills set', 'skills', 'workshops', 'key technologies', 'volunteer', 'technical expertise', 'languages and technologies', 'qualification', 'education/', 'competencies', 'skills:', 'academic credentials', 'technical skills', 'additional information', 'education:-', 'certification:', 'academic profile', 'education and certification', 'summary', 'certificates', 'certifications/licenses', 'skill set', 'education & certification', 'certifications', 'technical skill sets', 'software', 'seminars', 'educational', 'languages and tools', 'graduation', 'automation tools', 'objective', 'education / certification', 'academic background', 'achievements', 'technology and tools', 'skill matrix', 'tools and technology', 'technical conferences', 'projects overview', 'online profile links', 'responsibilities', 'technical set', 'academic projects', 'work', 'skills acquired', 'training', 'certification', 'certificate', 'presentations', 'technical', 'skills summary', 'tools', 'education:', 'portfolio', 'status', 'trainings', 'technical summary', 'technical proficiencies', 'academic', 'qualifications', 'publications', 'technical proficiency', 'awards', 'academic qualification', 'hobbies', 'groups', 'programming', 'technical skill set', 'rewards and achievements', 'core competencies', 'areas of specialization', 'academic qualifications', 'education /certification', 'projects', 'education/certification', 'technologies', 'detailed tech stack', 'education', 'technical sets', 'corporate', 'professional experience', 'languages', 'personal', 'technology summary', 'areas of experitse', 'military', 'certifications:']

    mwetokenizer = MWETokenizer(separator=' ')
    for each_word in exclude_list:
        mwetokenizer.add_mwe(each_word.split(" "))

    first_block = []

    for index,each_line in enumerate(text_content.split('\n')[:15]):

        each_line = each_line.lower()
        tokens = wordpunct_tokenize(each_line)
        tokens = mwetokenizer.tokenize(tokens)
        if tokens != [] :
            if tokens[0].strip() in exclude_list:
                first_block = text_content.split('\n')[:index]
                break
            else:
                first_block =  text_content.split('\n')[:15]

 #   print(first_block)
    return first_block
email_pattern = re.compile(r'([\w\-\.]+@(\w[\w\-]+\.)+[\w\-]+)',re.IGNORECASE)
linkedin_pattern = re.compile(r'((ftp|http|https):\/\/?)?((www|\w\w)\.)?linkedin.com(\w+:{0,1}\w*@)?(\S+)(:([0-9])+)?(\/|\/([\w#!:.?+=&%@!\-\/]))?')

def feeder_certificate(file_path, text_content):
#    # Uncomment for standalone testing
#    text_content = conv_tika_to_text(file_path)

    # Determine delimiter for sections
    text_temp_replaced = re.sub(r'\n \n', r'\n\n', text_content)
    text_temp = re.sub(r'\n{2,}', r'||', text_temp_replaced)
    # Split text content to list
    text_list = text_temp.split('||')

    # Sometimes tika is not able to split rows, then we collapse the delimiter.
    if len(text_list) < 3:
        text_temp = re.sub(r'\n{1,}', r'||', text_temp_replaced)
        # Split text content to list
        text_list = text_temp.split('||')

    text_part = ''

    '''
    First we will capture certification block only.
    If that is not found then we will search for certifi keyword and add a fuzzy logic to it.

    '''
    ''' PART 1:  '''

    tag = 'certifications'
    certification_headers = ['certifications:', 'certifications', 'certification:', 'certification']
    notags = ['education & certification', 'education/certification', 'education and certification', 'education / certification', 'education/ certification', 'education /certification',
              'for certification', 'in certification', 'on certification', 'get certifi', 'with certifi', 'become certifi', 'and certifications', '& certifications']

    # Capture certification block
    for idi, i in enumerate(text_list):
        if i.lower().find(tag) > -1:
            if any(nt in i.lower() for nt in notags):
                #print("Skipping because of misleading tags (notags)..")
                continue
            else:
                # if list element is the header
                if len(clean_multi_spaces_alphanum(i)) <= 15:
                    text_part = text_list[idi+1]
                    break
                else:
                    text_part = text_list[idi]
                    break

    # Clean certification block captured above
    text_part = re.sub(r'\uf0b7', r'\n', text_part)
    text_part = re.sub(r'\n{1,}', r'||', text_part)

    out_block = []
    unformatted_out = []
    if text_part != '':
        text_part_list = text_part.split('||')
        # Remove certification tag and any previous junks
        for idi, i in enumerate(text_part_list):
            if i.lower().find(tag) > -1:
                text_part_list = text_part_list[idi+1:]
        # Clean values
        for idi, i in enumerate(text_part_list):
            out_block.append(clean_multi_spaces_alphanum(i))

    # Save unformatted output for subtraction in education block.
    unformatted_out = out_block



    ''' PART 2:  '''

    if len(out_block) == 0:
        #print("Could not find certification block. Using tag search..")
        tag = 'certifi'
        text_part_list = []
        out_block = []
        unformatted_out = []
        # Make split more granular since we need sentences and not words
        text_temp = re.sub(r'\n{1,}', r'||', text_temp_replaced)
        # Split text content to list
        text_list = text_temp.split('||')

        # Find all elements with keyword matches.
        for idx, i in enumerate(text_list):
            if i.lower().find(tag) > -1:
                text_part_list.append(i)

        # Remove notags from list
        popid=[]
        for idi, i in enumerate(text_part_list):
            if any(nt in i.lower() for nt in notags):
                popid.append(idi)
        text_part_list = [i for j, i in enumerate(text_part_list) if j not in popid]

        # Remove original header (if any) from list
        popid=[]
        for idi, i in enumerate(text_part_list):
            if any(c ==clean_multi_spaces_alphanum(i).lower() for c in certification_headers):
                popid.append(idi)
        text_part_list = [i for j, i in enumerate(text_part_list) if j not in popid]

        # Remove huge elements if certification is captured in some other contexts
        popid=[]
        for idi, i in enumerate(text_part_list):
            if len(i) > 100:
                popid.append(idi)
        text_part_list = [i for j, i in enumerate(text_part_list) if j not in popid]

        # Clean values
        for idi, i in enumerate(text_part_list):
            temp = re.sub(r'\uf0b7', r'', i)
            out_block.append(clean_multi_spaces_alphanum(temp))

        # Save unformatted output for subtraction in education block.
        unformatted_out = out_block


        ''' PART 2.1:     Fuzzy Logic to combine duplicates'''
        if len(out_block) > 1:
            # Use fuzzy logic to remove duplicates
            df = pd.DataFrame(index = out_block, columns = out_block)
            for i in out_block:
                for j in out_block:
                    df.loc[i, j] = fuzz.partial_ratio(i,j)
            df = df.where(np.triu(np.ones(df.shape)).astype(np.bool))
            df = df.stack().reset_index()
            df.columns = ['Row','Column','Value']
            fuzzy_cut_off = 96
            sim_words = []
            for i in range(len(df)):
                if (df.loc[i,'Row'] != df.loc[i,'Column']) and (df.loc[i,'Value'] > fuzzy_cut_off):
                    sim_words.append((df.loc[i,'Row'], df.loc[i,'Column']))
            remove_sim_words = []
            for i in sim_words:
                if len(i[0]) >= len(i[1]):
                    remove_sim_words.append(i[1])
                else:
                    remove_sim_words.append(i[0])
            # Remove the elements from final list
            for i in remove_sim_words:
                out_block.remove(i)


    # Drop any list element which is blank
    if len(out_block) > 1:
        popid = []
        for idi, i in enumerate(out_block):
            if len(i) <= 1:
                popid.append(idi)
        out_block = [i for j, i in enumerate(out_block) if j not in popid]

    # return formatted output
    bullet = '• '
    if len(out_block) > 1:
        out_block = [bullet+item for item in out_block]

    if len(out_block) > 0:
        out = '\n'.join(out_block)
#        print(out)
#        print(unformatted_out)
        return [{"inference":resume_output_cleaner(out, "certificate"), "unformatted_inf":resume_output_cleaner(unformatted_out, "certificate")}]
    else:
        #print("Not Found")
        return [{"inference": ''}]

def feeder_linked_in(file_path, text_content):

    # original_text = text_content.split('\n')
    original_text = get_first_block_from_resume(text_content)
    new_text_list = []
    email_matches = []
    linkedin_matches = []
    for each_line in original_text:
         if each_line != '' :
            new_text_list.append(each_line.replace ('\t',' ').encode('ascii','ignore').decode('utf-8'))
    for sentence in new_text_list:
        if email_pattern.search(sentence):
            match = email_pattern.search(sentence)
            if match.group(0) not in email_matches:
                email_matches.append(match.group(0))
        if linkedin_pattern.search(sentence):
            match = linkedin_pattern.search(sentence)

            if match.group(0) not in linkedin_matches :
                linkedin_matches.append(match.group(0))

    final_output = []
    for each_answer in linkedin_matches:
        if "/in/" in each_answer:
            each_answer = resume_output_cleaner(each_answer,"linkedin")
            final_output.append({'inference':each_answer.strip()})

    return final_output


def feeder_email(file_path, text_content):

    original_text = text_content.lower()
    original_text = get_first_block_from_resume(original_text)
    new_text_list = []
    email_matches = []
    # linkedin_matches = []
    for each_line in original_text:
        if each_line != '' :
            new_text_list.append(each_line.replace ('\t',' ').encode('ascii','ignore').decode('utf-8'))
    for sentence in new_text_list:
        if email_pattern.search(sentence):
            match = email_pattern.search(sentence)
            if match.group(0) not in email_matches:
                email_matches.append(match.group(0))

    email_matches = list(set(email_matches))
    final_output = []
    for each_answer in list(set(email_matches)):
        each_answer = resume_output_cleaner(each_answer,"candidate_email")
        # final_output.append({'inference':each_answer.strip()})
        final_output.append(each_answer.strip())
    # print(final_output)
    output_list = [{'inference':str(",".join(s.strip() for s in final_output))}]
    # print(output_list)
    # return final_output
    return output_list


def feeder_phone(file_path,text_content):
    text_content = "\n".join(get_first_block_from_resume(text_content))

    text_content = '\n'.join(get_cleaned_encoded_text_from_jd(text_content))

    pattern = re.compile(r'[(+]?\d+[)]?[ \t\f\v\r]*[.-]?[ \t\v\f\r]*[(]?\d{2,}[)]?[ \t\v\f\r]*[.-]?[ \t\f\v\r]*[(]?\d{2,}[)]?[ \t\f\r\v]*[.-]?[ \t\v\f\r]*[(]?\d+[)]?[ \t\v\r\f]*')
    extractor = URLExtract()
    email_pattern = re.compile(r'([\w\-\.]+@(\w[\w\-]+\.)+[\w\-]+)')

    extractor = URLExtract()
    urls = extractor.find_urls(text_content)

    if len(urls) > 0:
        for url in urls:
            text_content = text_content.replace(url,' ')

    text_content = email_pattern.sub(' ',text_content)
    # print(text_content)

    answer = []
    result = re.findall(pattern,text_content)
    for each_match in result:
        phone_no_only_digits = []
        each_match_split = list(each_match)
        for each in each_match_split:
            string = ''
            if each in ['1','2','3','4','5','6','7','8','9','0']:
                phone_no_only_digits.append(each)
            else:
                continue
        if len(phone_no_only_digits) >= 10:
            for each in phone_no_only_digits:
                string = string + each

            each_match_sp_char_removed = re.sub(r'[-()+_]',' ',each_match)
            split_with_space = each_match_sp_char_removed.split()
            # print(split_with_space)
            phone_num_cleaned = each_match
            if len(split_with_space) > 3:
                phone_num_cleaned = ''
                for each_chunk_of_numbers in split_with_space:
                    no_of_num_in_chunk = list(each_chunk_of_numbers)
                    # print(no_of_num_in_chunk)
                    if len(no_of_num_in_chunk) <= 4:
                        phone_num_cleaned = phone_num_cleaned + " " + each_chunk_of_numbers

            answer.append(phone_num_cleaned)
    # print(answer)
    output = list(set(answer))
    final_output = []
    for each_answer in output:
        each_answer = resume_output_cleaner(each_answer,"candidate_phone_number")
        # final_output.append({'inference':each_answer})
        final_output.append(each_answer)
    # print(final_output)
    output_list = [{'inference':str(",".join(s.strip() for s in final_output))}]
 #  print(output_list)
    # return final_output
    return output_list

#searching whether file exists and return path of file
#def find(name, path):
 #   for root, dirs, files in os.walk(path):
  #      if name in files:
   #         return os.path.join(root, name)

df_csv = pd.read_csv('/home/drive/Downloads/ground_truth_2.csv', index_col=False)
path='/home/drive/Downloads/gr_resumes'
#name = input("Enter your resume name : ")
#parse=find(name,path)
j=0

#RESULTS after testing
def result_list_name():
    result_list_name=[]
    for root, dirs, files in os.walk(path):
        for file in files:
            if os.path.isfile(os.path.join(path,file)) and '.~lock.' in file:
                continue
            parse=os.path.join(root, file)
            parsed = unpack.from_file(parse)
            text_content = parsed['content']
            result_list_name.append(file)
        return result_list_name
n=result_list_name()

#RESULTS after testing
def result_list_phone():
    result_list_phone=[]
    for root, dirs, files in os.walk(path):
        for file in files:
            if os.path.isfile(os.path.join(path,file)) and '.~lock.' in file:
                continue
            parse=os.path.join(root, file)
            parsed = unpack.from_file(parse)
            text_content = parsed['content']
            phone_no_extracted=feeder_phone('',text_content)
            ph_num_extract=phone_no_extracted[0]['inference']

            result_phone_no=compare_phone(file,ph_num_extract)
            result_list_phone.append(result_phone_no)

     #   print(len(result_list_phone))
        return result_list_phone
p=result_list_phone()
#print(p)
phone_correct=0

# To count the no of correct matches
for i in p:
    if i==1:
        phone_correct+=1
print(phone_correct)

#RESULTS after testing
def result_list_email():
    result_list_email=[]
    for root, dirs, files in os.walk(path):
        for file in files:
            if os.path.isfile(os.path.join(path,file)) and '.~lock.' in file:
                continue
            parse=os.path.join(root, file)
            parsed = unpack.from_file(parse)
            text_content = parsed['content']
            email_extracted=feeder_email('',text_content)
            email_extract=email_extracted[0]['inference']

            result_email=compare_email(file,email_extract)
           # re=int(result_email)
            result_list_email.append(result_email)

      #  print(len(result_list_email))
        return result_list_email
e=result_list_email()
#print(e)

# To count the no of correct matches
email_correct=0
for i in e:
    if i==1:
        email_correct+=1
print(email_correct)


#RESULTS after testing
def result_list_linked_in():
    result_list_linked_in=[]
    for root, dirs, files in os.walk(path):
        for file in files:
            if os.path.isfile(os.path.join(path,file)) and '.~lock.' in file:
                continue
            parse=os.path.join(root, file)
            parsed = unpack.from_file(parse)
            text_content = parsed['content']
            linked_in=feeder_linked_in('',text_content)

            result_linked_in=compare_linked_in(file,linked_in)

            result_list_linked_in.append(result_linked_in)

      #  print(len(result_list_linked_in))
        return result_list_linked_in
l=result_list_linked_in()
#print(l)

# To count the no of correct matches
linked_in_correct=0
for i in l:
    if i==1:
        linked_in_correct+=1
print(linked_in_correct)


# results after certificate testing
def result_list_certificate():
    result_list_certificate=[]
    for root, dirs, files in os.walk(path):
        for file in files:
            if os.path.isfile(os.path.join(path,file)) and '.~lock.' in file:
                continue
            parse=os.path.join(root, file)
            parsed = unpack.from_file(parse)
            text_content = parsed['content']
            certificate_extracted=feeder_certificate('',text_content)
            certficate=certificate_extracted[0]['inference']

            result_certificate=compare_certificate(file,certficate)

            result_list_certificate.append(result_certificate)

      #  print(len(result_list_email))
        return result_list_certificate
c=result_list_certificate()
#print(c)
certificate_correct=0

# To count the no of correct matches
for i in c:
    if i==1:
        certificate_correct+=1
print(certificate_correct)


# USING FLAGS to display the data frame

flags = {
    'ph_no_flag':1,
    'email_flag':1,
    'linked_in_flag':0,
    'certificate_flag':0
     }

#single condition
if flags['ph_no_flag']==1 and flags['email_flag']==0 and flags['linked_in_flag']==0 and flags['certificate_flag']==0:
    df_phone = {'phone':p, 'resume':n }
    data_phone=pd.DataFrame(df_phone)
    print(data_phone.to_string(index=False))
    print("\n\n")

if flags['ph_no_flag']==1 and flags['email_flag']==1 and flags['linked_in_flag']==0 and flags['certificate_flag']==0:
    df_phone_email = {'phone':p, 'resume':n , 'email':e }
    data_phone_email=pd.DataFrame(df_phone_email)
    print(data_phone_email.to_string(index=False))
    print("\n\n")

if flags['ph_no_flag']==1 and flags['email_flag']==0 and flags['linked_in_flag']==1 and flags['certificate_flag']==0:
    df_phone_email = {'phone':p, 'resume':n , 'linked_in':l }
    data_phone_email=pd.DataFrame(df_phone_email)
    print(data_phone_email.to_string(index=False))
    print("\n\n")

if flags['ph_no_flag']==1 and flags['email_flag']==0 and flags['linked_in_flag']==0 and flags['certificate_flag']==1:
    df_phone_email = {'phone':p, 'resume':n ,  'certificates': c }
    data_phone_email=pd.DataFrame(df_phone_email)
    print(data_phone_email.to_string(index=False))
    print("\n\n")

if flags['email_flag']==1 and flags['ph_no_flag']==0 and flags['linked_in_flag']==0 and flags['certificate_flag']==0:
    df_email = {'email':e , 'resume':n }
    data_email = pd.DataFrame(df_email)
    print(data_email.to_string(index=False))
    print("\n\n")


if flags['email_flag']==1 and flags['ph_no_flag']==0 and flags['linked_in_flag']==1 and flags['certificate_flag']==0:
    df_email_linked_in = {'email':e , 'resume':n , 'linked_in':l }
    data_email_linked_in = pd.DataFrame(df_email_linked_in)
    print(data_email_linked_in.to_string(index=False))
    print("\n\n")

if flags['email_flag']==1 and flags['ph_no_flag']==0 and flags['linked_in_flag']==0 and flags['certificate_flag']==1:
    df_email_certificate = {'email':e , 'resume':n , 'certificates':c }
    data_email_certificate = pd.DataFrame(df_email_certificate)
    print(data_email_certificate.to_string(index=False))
    print("\n\n")


if flags['linked_in_flag']==1 and flags['ph_no_flag']==0 and flags['email_flag']==0 and flags['certificate_flag']==0 :
    df_linked_in= {'linked_in':l , 'resume':n }
    data_linked_in =pd.DataFrame(df_linked_in)
    print(data_linked_in.to_string(index=False))
    print("\n\n")


if flags['linked_in_flag']==1 and flags['ph_no_flag']==0 and flags['email_flag']==0 and flags['certificate_flag']==1 :
    df_linked_in= {'linked_in':l , 'resume':n , 'certificates':c }
    data_linked_in =pd.DataFrame(df_linked_in)
    print(data_linked_in.to_string(index=False))
    print("\n\n")

if flags['certificate_flag']==1 and flags['ph_no_flag']==0 and flags['linked_in_flag']==0 and flags['email_flag']==0:
    df_certificate = {'certificate':c , 'resume':n }
    data_certificate =pd.DataFrame(df_certificate)
    print(data_certificate.to_string(index=False))
    print("\n\n")






2020-01-07 16:08:17,077 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.16/tika-server-1.16.jar to /tmp/tika-server.jar.
